<a href="https://colab.research.google.com/github/peeush-agarwal/week-based-learning/blob/master/MNIST_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural network for MNIST dataset

MNIST (National Institute of Standards and Technology) provides us cleaned dataset of digits (0-9).

## Import libraries

In [0]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

## Load dataset from PyTorch library

In [0]:
root_dir = './Data'
download = True
batch_size = 100
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,))])

trainset = datasets.MNIST(root=root_dir,train=True,transform=transform,download=download)
testset = datasets.MNIST(root=root_dir, train=False, transform=transform, download=download)

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [0]:
classes = trainset.classes
print(classes)

In [0]:
print(f'Train size: {len(trainset)}')
print(f'Test size: {len(testset)}')

## View random images from dataset

In [0]:
inputs, classes = next(iter(trainloader))
#print(type(inputs))
#print(type(classes))
#print(inputs.shape)

plt.figure(figsize=(10,10))
for idx, (img, cls) in enumerate(zip(inputs, classes)):
    plt.subplot(10,10,idx+1)
    plt.imshow(img.numpy().squeeze()) #Flatten the image matrix
    #plt.title(f'Class:{cls}')
    plt.axis('off')
plt.show()

## Build a neural network

In [0]:
class BasicNN(nn.Module):
    def __init__(self, inputs, outputs):
        super(BasicNN, self).__init__()
        self.fc1 = nn.Linear(inputs, 10) # Hidden layer with 10 neurons
        self.fc2 = nn.Linear(10, outputs) # Output layer
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        
        return out

In [0]:
device = torch.device('cuda')

input_size = 28 * 28
output_size = 10
net = BasicNN(input_size, output_size).to(device) # Image size = 28*28, Output classes = 10

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [0]:
def train(num_epochs):
    loss_list = []
    for epoch in range(num_epochs):
        for batch, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.view(-1, input_size)

            optimizer.zero_grad()

            outputs = net(inputs) # Forward propogation

            loss = loss_func(outputs, labels) # Calculate loss

            loss.backward() # Back propogation

            optimizer.step()
        loss_list.append(loss)
    return loss_list

In [0]:
def evaluate():
    accuracy_list = []
    for batch, (images, labels) in enumerate(testloader):
        total = 0
        correct = 0
        images = images.to(device)
        labels = labels.to(device)
        images = images.view(-1, input_size)
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        total += len(images)
        correct += (preds == labels).sum()

        accuracy_list.append(100.0*correct/total)
    return accuracy_list

In [0]:
def plot(train_loss_list, eval_acc_list):
    plt.subplot(121)
    plt.plot(range(len(train_loss_list)), train_loss_list)
    plt.xlabel('Epochs')
    plt.ylabel('Training Loss')
    plt.title('Training Loss over epochs')
    
    plt.subplot(122)
    plt.plot(range(len(eval_acc_list)), eval_acc_list)
    plt.xlabel('Batch (100)')
    plt.ylabel('Test Accuracy')
    plt.title('Test Accuracy over batches of 100')
    
    plt.show()

In [0]:
for i, num_epochs in enumerate([2, 5, 10, 20, 50, 100]):
    print(f'Epochs={num_epochs}')
    plot(train(num_epochs),evaluate())